## Create the Sun Cloud Routes Layer

The Sun Cloud Routes layer delineates which roadways are included in the Sun Cloud data portal.

Select all roads that meet functional class requirements or are part of Kyle's manual review of regionally significant routes.
* Arterial or above
* Major collectors in ubran areas outside of Phoenix and Tucson

Uses the default environment workspace for temp layers.

**routes.gdb** contains important imputs into the process and should not be deleted!

| Code | Description |
| ---- | ----------- |
|1|Rural Principal Arterial - Interstate|
|2|Rural Principal Arterial - Ohter|
|3|Rural Principal Arterial - Other Fwys & Expwys|
|6|Rural Minor Arterial|
|7|Rural Major Collector|
|8|Rural Minor Collector|
|9|Rural Local|
|11|Urban Principal Arterial - Interstate|
|12|Urban Principal Arterial - Other Pwys & Expwys|
|14|Urban Principal Arterial - Other|
|16|Urban Minor Arterial|
|19|Urban Local|
|17|Urban Major Collector|
|18|Urban Minor Collector|


### Jurisdictional Boundaries

As needed, download and extract the geographic boundaries layers from

City boundaries
https://azgeo-open-data-agic.hub.arcgis.com/datasets/azgeo::incorporated-city-boundaries-1/explore?layer=2&location=34.134590%2C-111.945292%2C7.96
 
MPO boundaries
https://azgeo-open-data-agic.hub.arcgis.com/datasets/AZMAG::arizona-metropolitan-planning-organizations/explore?location=33.331142%2C-112.462165%2C8.50
 
COG boundaries
https://azgeo-open-data-agic.hub.arcgis.com/datasets/AZMAG::arizona-councils-of-government/explore?location=34.135148%2C-111.914470%2C7.95
 

In [3]:
import arcpy
import os
import re

# begin by adding layers below to map and setting the appropriate variable values below

# specify layer names

# ATIS layers - https://azgeo-open-data-agic.hub.arcgis.com/maps/azgeo::adot-highway-performance-monitoring-system-hpms-2020-data-/about

# ATIS Routes - download the AllRoadsNetwork layer from:
# https://azgeo-open-data-agic.hub.arcgis.com/datasets/azgeo::adot-highway-performance-monitoring-system-hpms-2020-data-/
#atis_routes = 'AllRoadsNetwork'
# actually - this layer doesn't get used. the Functional Class is the basis


# ATIS FacilityType
# https://azgeo-open-data-agic.hub.arcgis.com/datasets/azgeo::adot-highway-performance-monitoring-system-hpms-2020-data-/about?layer=17
atis_facility_type = 'FacilityType'

# 2010 Census Urban Areas - https://hub.arcgis.com/maps/esri::usa-urban-areas/

target_gdb = 'routes.gdb'

In [4]:
def select_pattern(feature_layer, field, pattern, select_type = "NEW SELECTION"):
    matcher = re.compile(pattern)
    describe_feature = arcpy.Describe(feature_layer)
    
    oid_list = []
    with arcpy.da.SearchCursor(feature_layer, ['OID@', field]) as sc:
        for row in sc:
            if matcher.search(row[1]) is not None:
                oid_list.append(row[0])
    if(len(oid_list)) == 0:
        print('No records matched.')
        return None
    
    sql = "\"{}\" IN ({})".format(
        describe_feature.OIDFieldName,
        ",".join(["{}".format(oid) for oid in oid_list])
    )

    arcpy.SelectLayerByAttribute_management(
        feature_layer,
        select_type,
        sql
    )

First, export all online feature layers to a local filegeodatabase for acceptable performance.

In [ ]:
# ATIS Functional CLass
# from https://azgeo-open-data-agic.hub.arcgis.com/datasets/azgeo::adot-functional-classification/about
# https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Functional_Classification_2020_Test/FeatureServer/0/
online_fc = 'FunctionalClassification'

# IMPORTANT - make sure only "current" features get copied!
arcpy.management.SelectLayerByAttribute(
    online_fc, 
    "NEW_SELECTION", 
    "ToDate IS NULL", 
    None
)

In [ ]:
# copy local
arcpy.conversion.ExportFeatures(
    online_fc, 
    "atis_functional_class"
)

In [ ]:
# Facility Type no longer seems to be on AZ Geo. Using offline copy.
arcpy.conversion.ExportFeatures(
    r'atis_facilitytype.gdb\FacilityType', 
    "atis_facility_type"
)

In [12]:
# remove ramps - may want to revisit this someday
arcpy.management.SelectLayerByAttribute(
    "atis_facility_type", 
    "NEW_SELECTION", 
    "FacilityType IN (4, 5, 7)"
)

<Result 'atis_facility_type'>

In [13]:
arcpy.conversion.ExportFeatures(
    "atis_facility_type", 
    r"routes.gdb\atis_remove_ramps"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\routes.gdb\\atis_remove_ramps'>

In [72]:
# clip functional class layer to sun cloud counties
arcpy.analysis.PairwiseClip(
    "atis_functional_class", 
    r'routes.gdb\counties', 
    'atis_fclass_clipped', 
    None
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\sun_cloud_routes.gdb\\atis_fclass_clipped'>

In [73]:
# spatially join the name of the urban area
arcpy.analysis.SpatialJoin(
    'atis_fclass_clipped', 
    r'routes.gdb\USA_Urban_Areas', 
    'func_class_clipped_urban_area', 
    "JOIN_ONE_TO_ONE", 
    "KEEP_ALL"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\sun_cloud_routes.gdb\\func_class_clipped_urban_area'>

In [74]:
# select routes that overlap with Kyle's regionally significant routes
arcpy.management.SelectLayerByLocation(
    "func_class_clipped_urban_area", 
    "HAVE_THEIR_CENTER_IN", 
    r"routes.gdb\regionally_significant_routes", 
    "1 Meters", 
    "NEW_SELECTION", 
    "NOT_INVERT"
)

<Result 'func_class_clipped_urban_area'>

In [75]:
# select by functional class
arcpy.management.SelectLayerByAttribute(
    'func_class_clipped_urban_area', 
    "ADD_TO_SELECTION", 
"""
FunctionalClass IN (1, 2, 3, 6, 11, 12, 14, 16)
OR (
  NAME NOT IN ('Tucson, AZ', 'Phoenix--Mesa, AZ', 'Avondale--Goodyear, AZ', 'Buckeye, AZ') 
  AND FunctionalClass IN (7, 17)
)
"""
)

<Result 'func_class_clipped_urban_area'>

In [76]:
# export selected features
arcpy.conversion.ExportFeatures(
    'func_class_clipped_urban_area', 
   "fc2"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\sun_cloud_routes.gdb\\fc2'>

In [77]:
arcpy.management.AddJoin(
    "fc2", 
    "FunctionalClass", 
    "fclass_crosswalk.csv", 
    "atis_code"
)

<Result 'fc2'>

## Remove ramps

In [78]:
arcpy.management.SelectLayerByLocation(
    "fc2", 
    "SHARE_A_LINE_SEGMENT_WITH", 
    r"routes.gdb\atis_remove_ramps",
    None, 
    "NEW_SELECTION", 
    "INVERT"
)

<Result 'fc2'>

## Copy over all routes then pare down to just what we want for SC

In [79]:
# export selected features
arcpy.conversion.ExportFeatures(
    'fc2', 
    "all_routes_no_ramps"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\sun_cloud_routes.gdb\\all_routes_no_ramps'>

### Remove extraneous features (including non-inventory direction routes on arterials

In [80]:
# remove all 00 ... 0 routes
pattern = '^00.{28}0\s*$' # Match 00*0
select_pattern("all_routes_no_ramps", 'RouteId', pattern)
# after selecting, delete...

In [81]:
arcpy.management.DeleteFeatures("all_routes_no_ramps")

<Result 'all_routes_no_ramps'>

In [ ]:
# cells below disabled ... have a new way to handle this!

In [82]:
# remove remaining interstate ramps
arcpy.management.SelectLayerByAttribute(
    "all_routes_no_ramps", 
    "CLEAR_SELECTION"
)
pattern = '\s{1,4}[ISU]\s\d{6}[A-Z]'
select_pattern("all_routes_no_ramps", 'RouteId', pattern)

In [83]:
arcpy.management.DeleteFeatures("all_routes_no_ramps")

<Result 'all_routes_no_ramps'>

In [84]:
arcpy.analysis.PairwiseDissolve(
    "all_routes_no_ramps", 
    "all_routes_dissolve", 
    "RouteId;federal_code", None, "MULTI_PART"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\sun_cloud_routes.gdb\\all_routes_dissolve'>

In [85]:
# finally, remove dangles - run below then delete features
arcpy.management.SelectLayerByLocation(
    "all_routes_dissolve", 
    "WITHIN", 
    r"routes.gdb\dangles", 
    None, "NEW_SELECTION"
)

<Result 'all_routes_dissolve'>

In [86]:
# delete dangles
arcpy.management.DeleteFeatures("all_routes_dissolve")

<Result 'all_routes_dissolve'>

### delete more extraneous routes

In [87]:
arcpy.management.SelectLayerByAttribute(
    "all_routes_dissolve", 
    "ADD_TO_SELECTION", 
    "Shape_Length < 25", 
    None
)

<Result 'all_routes_dissolve'>

In [88]:
arcpy.management.SelectLayerByAttribute(
    "all_routes_dissolve", 
    "ADD_TO_SELECTION", 
    """RouteId LIKE '  I%'
AND
Shape_Length < 2500""", 
    None
)

<Result 'all_routes_dissolve'>

In [89]:
arcpy.management.DeleteFeatures('all_routes_dissolve')

<Result 'all_routes_dissolve'>

### deal with intermittent divided overlaps

In [90]:
# deal with routes that are intermittently divided
# in the below, we remove the overlapping sections of lines for
# the divided sections of intermittently divided routes

In [91]:
#pattern = '\s{1,4}0\s$' # Match 0 at end
pattern = '\s{1,4}0[\s123]$'
select_pattern("all_routes_dissolve", 'RouteId', pattern)

In [92]:
arcpy.management.SelectLayerByAttribute(
    "all_routes_dissolve", 
    "ADD_TO_SELECTION", 
    """RouteId IN (
        '  SB008                       03'
    )"""
)

<Result 'all_routes_dissolve'>

In [93]:
arcpy.conversion.ExportFeatures(
    "all_routes_dissolve",  
    "routes_ending_in_zero"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\sun_cloud_routes.gdb\\routes_ending_in_zero'>

In [94]:
 arcpy.management.SelectLayerByAttribute(
     "all_routes_dissolve", 
     "SWITCH_SELECTION"
 )

<Result 'all_routes_dissolve'>

In [95]:
arcpy.conversion.ExportFeatures(
    "all_routes_dissolve", 
    "routes_not_ending_in_zero" 
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\sun_cloud_routes.gdb\\routes_not_ending_in_zero'>

In [96]:
arcpy.analysis.PairwiseErase(
    "routes_ending_in_zero", 
    "routes_not_ending_in_zero", 
    "routes_ending_in_zero_not_overlapping", 
    None
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\sun_cloud_routes.gdb\\routes_ending_in_zero_not_overlapping'>

In [109]:
arcpy.management.Merge(
    "routes_not_ending_in_zero;routes_ending_in_zero_not_overlapping", 
    "routes_non_overlap", 
    'RouteId "RouteId" true true false 32 Text 0 0,First,#,routes_not_ending_in_zero,RouteId,0,32,routes_ending_in_zero_not_overlapping,RouteId,0,32;federal_code "federal_code" true true false 4 Long 0 0,First,#,routes_not_ending_in_zero,federal_code,-1,-1,routes_ending_in_zero_not_overlapping,federal_code,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,routes_not_ending_in_zero,Shape_Length,-1,-1', 
    "NO_SOURCE_INFO"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\sun_cloud_routes.gdb\\routes_non_overlap'>

In [110]:
arcpy.management.SelectLayerByAttribute(
    "routes_non_overlap", 
    "NEW_SELECTION", 
    """RouteId IN (
    '  SB019                       01',
    '00  COTTON              LN    0E',
    '  S 087                       0 ',
    '  S 087                         ')"""
)



<Result 'routes_non_overlap'>

In [111]:
arcpy.management.DeleteFeatures("routes_non_overlap")

<Result 'routes_non_overlap'>

In [112]:
arcpy.management.Project(
    "routes_non_overlap", 
    "fc3", 
    'PROJCS["NAD_1983_StatePlane_Arizona_Central_FIPS_0202_Feet_Intl",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",700000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-111.9166666666667],PARAMETER["Scale_Factor",0.9999],PARAMETER["Latitude_Of_Origin",31.0],UNIT["Foot",0.3048]]',
    "WGS_1984_(ITRF00)_To_NAD_1983", 'PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]', 
    "NO_PRESERVE_SHAPE", 
    None, 
    "NO_VERTICAL"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\sun_cloud_routes.gdb\\fc3'>

### dissolve by Route ID and then join functional class back - down to 1350 features

In [113]:
# dissolve by route id
arcpy.analysis.PairwiseDissolve(
    "fc3", 
    "sun_cloud_rout_PairwiseDisso", 
    "RouteId", None, "MULTI_PART", ''
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\sun_cloud_routes.gdb\\sun_cloud_rout_PairwiseDisso'>

In [114]:
# join the max functional class back to the dissolved features
#arcpy.management.AddSpatialJoin(
#    "sun_cloud_rout_PairwiseDisso", 
#    "fc3", 
#    "JOIN_ONE_TO_ONE", 
#    "KEEP_ALL", 
#    r'functional_class "Functional Classification" true true false 4 Long 0 0,Min,#,routes.gdb\sun_cloud_routes,functional_class,-1,-1', 
#    "SHARE_A_LINE_SEGMENT_WITH", None, ''
#)
arcpy.management.AddSpatialJoin(
    "sun_cloud_rout_PairwiseDisso", 
    "fc3", 
    "JOIN_ONE_TO_ONE", 
    "KEEP_ALL", 
    '', 
    "SHARE_A_LINE_SEGMENT_WITH", None, ''
)


<Result 'sun_cloud_rout_PairwiseDisso'>

In [115]:
# save the result
arcpy.conversion.ExportFeatures(
    "sun_cloud_rout_PairwiseDisso", 
    "fc4", 
    '', 
    "NOT_USE_ALIAS"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\sun_cloud_routes.gdb\\fc4'>

In [116]:
# manually fix FC on a few routes - consistent 2
arcpy.management.SelectLayerByAttribute(
    "fc4", 
    "NEW_SELECTION", 
    """RouteId IN (
        '  U 060                       0 ',
        '  U 060                         ',
        '  S 085                       02',
        '  S 085                        2',
        '07  NORTHERN            PKWY  0 '
    )"""
)

arcpy.management.CalculateField(
    "fc4", "federal_code", "2", 
    "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS"
)

arcpy.management.SelectLayerByAttribute(
    "fc4", "CLEAR_SELECTION", '', None
)

<Result 'fc4'>

In [117]:
# KEEP AND RENAME FIELDS
working_layer = "fc4"

system_fields = [arcpy.Describe(working_layer).OIDFieldName, 'Shape', 'geom', 'Shape_Length']

current_fields = [f.name for f in arcpy.ListFields(working_layer) if not f.name in system_fields]


# tuples of CurrentFieldName, target_field_name, Alias
target_fields = [
    ('RouteId', 'route_id', 'Route ID'),
    ('federal_code', 'functional_class', 'Functional Classification')
]

delete_fields = [field for field in current_fields if not field in [a for a, b, c in target_fields]]

# delete unused fields
if delete_fields:
    arcpy.management.DeleteField(working_layer, delete_fields)

# add check if we're just changing case ... if so, double rename
    
for current_name, new_name, alias in target_fields:
    # arcgis can't change capitalization ... a hacky workaround
    if new_name.lower() == current_name.lower():
        intermediate_name = current_name + "1"
        arcpy.management.AlterField(working_layer, current_name, intermediate_name)
        current_name = intermediate_name 
    arcpy.management.AlterField(working_layer, current_name, new_name, alias)

In [118]:
# add highway 87 back in
arcpy.management.Merge(
    r"fc4;routes.gdb\hwy_87_fix", 
    r"routes.gdb\routes", 
    r'route_id "Route ID" true true false 32 Text 0 0,First,#,fc4,route_id,0,32,routes.gdb\hwy_87_fix,route_id,0,32;functional_class "Functional Classification" true true false 4 Long 0 0,First,#,fc4,functional_class,-1,-1,routes.gdb\hwy_87_fix,functional_class,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,fc4,Shape_Length,-1,-1,routes.gdb\hwy_87_fix,Shape_Length,-1,-1', 
    "NO_SOURCE_INFO"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\routes\\routes.gdb\\routes'>

### Join Jurisdiction Boundaries